In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.image as mpl_img
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

from htc.cpp import map_label_image
from htc.models.image.DatasetImage import DatasetImage
from htc.settings_seg import settings_seg
from htc.tivita.DataPath import DataPath
from htc.utils.colors import lighten_color
from htc.utils.Config import Config
from htc.utils.helper_functions import median_table
from htc.utils.LabelMapping import LabelMapping
from htc_projects.species.settings_species import settings_species

pio.kaleido.scope.mathjax = None

In [2]:
label_mapping = settings_species.label_mapping
label_mapping.label_colors = settings_seg.label_colors_paper
config = Config({"label_mapping": label_mapping, "input/n_channels": 3})
path_physiological = DataPath.from_image_name("SPACE_000102#2020_12_17_19_00_25@semantic#primary")
path_malperfused = DataPath.from_image_name("SPACE_000106#2021_01_09_16_30_15@semantic#primary")
sample_physiological = DatasetImage([path_physiological], train=False, config=config)[0]
sample_malperfused = DatasetImage([path_malperfused], train=False, config=config)[0]

mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_physiological_rgb.png",
    sample_physiological["features"].numpy(),
)
mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_malperfused_rgb.png", sample_malperfused["features"].numpy()
)

mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_physiological_seg.png",
    map_label_image(sample_physiological["labels"].numpy(), label_mapping),
)
mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_malperfused_seg.png",
    map_label_image(sample_malperfused["labels"].numpy(), label_mapping),
)

In [3]:
label_mapping = LabelMapping(
    {"kidney": 0},
    label_colors={"kidney": settings_seg.label_colors_paper["kidney"], "unknown": "#00000000"},
)
config = Config({"label_mapping": label_mapping, "input/n_channels": 3})
sample_physiological = DatasetImage([path_physiological], train=False, config=config)[0]
mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_physiological_seg_kidney-only.png",
    map_label_image(sample_physiological["labels"].numpy(), label_mapping),
)

label_mapping.label_colors["kidney"] = lighten_color(settings_seg.label_colors_paper["kidney"], 0.65)
config = Config({"label_mapping": label_mapping, "input/n_channels": 3})
sample_malperfused = DatasetImage([path_malperfused], train=False, config=config)[0]
mpl_img.imsave(
    settings_species.paper_dir / "method_concept_example_malperfused_seg_kidney-only.png",
    map_label_image(sample_malperfused["labels"].numpy(), label_mapping),
)

In [4]:
df = median_table(paths=[path_physiological, path_malperfused])
wavelengths = np.arange(500, 1000, 5) + 2.5

spectrum_physiological = df[(df.image_name == path_physiological.image_name()) & (df.label_name == "kidney")].iloc[0][
    "median_normalized_spectrum"
]
spectrum_malperfused = df[(df.image_name == path_malperfused.image_name()) & (df.label_name == "kidney")].iloc[0][
    "median_normalized_spectrum"
]

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=wavelengths,
        y=spectrum_physiological,
        line_color=settings_seg.label_colors_paper["kidney"],
        name="physiological",
    )
)
fig.add_trace(
    go.Scatter(
        x=wavelengths,
        y=spectrum_malperfused,
        line_color=settings_seg.label_colors_paper["kidney"],
        line_dash="dot",
        name="malperfused",
    )
)

fig.update_layout(height=300, width=500, template="plotly_white")
fig.update_layout(font_family="Libertinus Sans", font_size=16)
fig.update_layout(margin=dict(l=0, r=0, t=10, b=0))
fig.update_layout(legend=dict(orientation="h", yanchor="top", y=1.17, xanchor="center", x=0.5))
fig.update_yaxes(title="<b>L1 normalized<br>reflectance [a.u.]</b>")
fig.update_xaxes(title="<b>wavelength [nm]</b>")

fig.write_image(settings_species.paper_dir / "method_concept_example_spectra_kidney.pdf")
fig